## Demonstrate **hklpy2**'s API

A working notebook as the package is being developed.
***Anything*** could change.

In [1]:
import datetime
import hklpy2
print(f"{datetime.datetime.now()}")
print(f"{hklpy2.__version__=}")
print(f"{hklpy2.solvers()=}")

2024-05-20 18:20:04.393039
hklpy2.__version__='0.0.10.dev27+g9d41cc4.d20240520'
hklpy2.solvers()={'hkl_soleil': 'hklpy2.backends.hkl_soleil:HklSolver', 'no_op': 'hklpy2.backends.no_op:NoOpSolver', 'th_tth': 'hklpy2.backends.th_tth_q:ThTthSolver'}


TODO: Make it easier to specify the pseudos, reals, and extras for any geometry & mode.

Can a given geometry assume (as a default?) that the first $p$ pseudos and the first $r$ reals (defined in the `DiffractometerBase` subclass) are the ones to use for the given Solver?  Here, $p$ and $r$ are the number of pseudos and reals, respectively, required by the geometry.  The order is important.  User could override.

In [2]:
# solver = hklpy2.solver_factory("no_op", "NoOpSolver ignores the gname")
solver = hklpy2.solver_factory("hkl_soleil", geometry="E4CV, hkl")
print(f"{solver=}")
print(f"{solver.pseudo_axis_names=!r}")
print(f"{solver.real_axis_names=}")

solver=HklSolver(name='hkl_soleil', version='v5.0.0.3434', geometry='E4CV, hkl', engine='')
solver.pseudo_axis_names=['h', 'k', 'l']
solver.real_axis_names=['omega', 'chi', 'phi', 'tth']


In [3]:
from hkl import SimulatedE4CV

e4cv = SimulatedE4CV("", name="e4cv")
print(f"{e4cv=!r}")

solver.auto_assign_axes(e4cv)
print(f"{solver.user_pseudos=}")
print(f"{solver.user_reals=}")
print(f"{solver.user_extras=}")

e4cv=SimulatedE4CV(prefix='', name='e4cv', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['h', 'h.readback', 'h.setpoint', 'k', 'k.readback', 'k.setpoint', 'l', 'l.readback', 'l.setpoint', 'omega', 'chi', 'phi', 'tth'], configuration_attrs=['energy', 'energy_units', 'energy_offset', 'geometry_name', 'class_name', 'sample_name', 'lattice', 'lattice_reciprocal', 'U', 'UB', 'reflections_details', 'ux', 'uy', 'uz', 'diffractometer_name', '_hklpy_version', '_pseudos', '_reals', '_constraints', '_mode', 'orientation_attrs', 'h', 'k', 'l'], concurrent=True)
solver.user_pseudos=(PseudoSingle(prefix='', name='e4cv_h', parent='e4cv', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['readback', 'setpoint'], configuration_attrs=[], idx=0), PseudoSingle(prefix='', name='e4cv_k', parent='e4cv', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['readback', 'setpoint'], configuration_attrs=[], 

In [4]:
tuple([axis.position for axis in e4cv.pseudo_positioners]), tuple([axis.position for axis in e4cv.real_positioners])

((0.0, 0.0, 0.0), (0, 0, 0, 0))

In [5]:
from ophyd import Component as Cpt
from ophyd import Kind
from ophyd import PseudoSingle
from ophyd import SoftPositioner

NORMAL_HINTED = Kind.hinted | Kind.normal

class Fourc(hklpy2.DiffractometerBase):
    """Test case."""

    h = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    omega = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind=NORMAL_HINTED)
    tth = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

    h2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    k2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741
    l2 = Cpt(PseudoSingle, "", kind=NORMAL_HINTED)  # noqa: E741

    psi = Cpt(SoftPositioner, limits=(-170, 170), init_pos=0, kind=NORMAL_HINTED)

fourc = Fourc("", name="fourc")
solver.auto_assign_axes(fourc)
print(f"{solver.user_pseudos=}")
print(f"{solver.user_reals=}")
print(f"{solver.user_extras=}")

solver.user_pseudos=(PseudoSingle(prefix='', name='fourc_h', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['readback', 'setpoint'], configuration_attrs=[], idx=0), PseudoSingle(prefix='', name='fourc_k', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['readback', 'setpoint'], configuration_attrs=[], idx=1), PseudoSingle(prefix='', name='fourc_l', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['readback', 'setpoint'], configuration_attrs=[], idx=2))
solver.user_reals=(SoftPositioner(name='fourc_omega', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(-180, 180), source='computed'), SoftPositioner(name='fourc_chi', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(-180, 180), source='computed'), SoftPositioner(name='fourc_phi', parent='fourc', settle_time=0.0, timeout=None, egu='', limits=(-180, 180), 

In [13]:
fourc.report

{'position': (0, 0, 0, 0, 0, 0, 0, 0, 0, 0)}

In [6]:
fourc.solver_name = "hkl_soleil"
fourc._solver.geometry = "E4CV, hkl"
fourc._solver.auto_assign_axes(fourc)
print(f"{len(fourc._solver.user_pseudos)=}")
print(f"{len(fourc._solver.user_reals)=}")
print(f"{len(fourc._solver.user_extras)=}")

len(fourc._solver.user_pseudos)=3
len(fourc._solver.user_reals)=4
len(fourc._solver.user_extras)=4


In [7]:
import math
vibranium_lattice = hklpy2.Lattice(2*math.pi)
print(f"{vibranium_lattice=}")

vibranium_lattice=Lattice(a=6.2832, b=6.2832, c=6.2832, alpha=90.0, beta=90.0, gamma=90.0)


In [8]:
vibranium = hklpy2.Sample(fourc._solver, vibranium_lattice, name="vibranium")
vibranium_lattice.digits = 2
print(f"{vibranium=}")

vibranium=Sample(name='vibranium', lattice=Lattice(a=6.28, b=6.28, c=6.28, alpha=90.0, beta=90.0, gamma=90.0))


In [9]:
vibranium.reflections.ordering = []

In [10]:
vibranium.reflections.add(
    hklpy2.Reflection(
        fourc._solver, 
        dict(h=1, k=0, l=0), 
        dict(omega=10, chi=0, phi=0, tth=20), 
        wavelength=1.00, 
        name="r1"
    )
)
vibranium.reflections.add(
    hklpy2.Reflection(
        fourc._solver, 
        dict(h=0, k=1, l=0), 
        dict(omega=10, chi=-90, phi=0, tth=20), 
        wavelength=1.00, 
        name="r2"
    )
)
for r in vibranium.reflections.ordering:
    print(f"{vibranium.reflections[r]}")

Reflection(name='r1', geometry='E4CV, hkl', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r2', geometry='E4CV, hkl', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [11]:
vibranium.reflections.swap()
print(f"{vibranium.reflections.ordering=}")
for r in vibranium.reflections.ordering:
    print(f"{vibranium.reflections[r]}")

vibranium.reflections.ordering=['r2', 'r1']
Reflection(name='r2', geometry='E4CV, hkl', pseudos={'h': 0, 'k': 1, 'l': 0}, angles={'omega': 10, 'chi': -90, 'phi': 0, 'tth': 20}, wavelength=1.0)
Reflection(name='r1', geometry='E4CV, hkl', pseudos={'h': 1, 'k': 0, 'l': 0}, angles={'omega': 10, 'chi': 0, 'phi': 0, 'tth': 20}, wavelength=1.0)


In [12]:
fourc._solver.geometries

['E4CH, emergence',
 'E4CH, hkl',
 'E4CH, incidence',
 'E4CH, psi',
 'E4CH, q',
 'E4CV, emergence',
 'E4CV, hkl',
 'E4CV, incidence',
 'E4CV, psi',
 'E4CV, q',
 'E6C, emergence',
 'E6C, hkl',
 'E6C, incidence',
 'E6C, psi',
 'E6C, q2',
 'E6C, qper_qpar',
 'E6C, tth2',
 'ESRF ID01 PSIC, hkl',
 'K4CV, emergence',
 'K4CV, eulerians',
 'K4CV, hkl',
 'K4CV, incidence',
 'K4CV, psi',
 'K4CV, q',
 'K6C, emergence',
 'K6C, eulerians',
 'K6C, hkl',
 'K6C, incidence',
 'K6C, psi',
 'K6C, q2',
 'K6C, qper_qpar',
 'K6C, tth2',
 'PETRA3 P09 EH2, hkl',
 'PETRA3 P23 4C, emergence',
 'PETRA3 P23 4C, hkl',
 'PETRA3 P23 4C, incidence',
 'PETRA3 P23 4C, q2',
 'PETRA3 P23 4C, qper_qpar',
 'PETRA3 P23 4C, tth2',
 'PETRA3 P23 6C, emergence',
 'PETRA3 P23 6C, hkl',
 'PETRA3 P23 6C, incidence',
 'PETRA3 P23 6C, psi',
 'PETRA3 P23 6C, q2',
 'PETRA3 P23 6C, qper_qpar',
 'PETRA3 P23 6C, tth2',
 'SOLEIL MARS, emergence',
 'SOLEIL MARS, hkl',
 'SOLEIL MARS, incidence',
 'SOLEIL MARS, psi',
 'SOLEIL MARS, q',
 'SOL